#---------------------**CROWD ANOMALY DETECTION**---------------------

> ### Presented by 
> # Vishakha Bhat and Sambit Sanyal


># ****DISCLAIMER****
> ### This code is best run using Google colab. Thats where it was tried and tested

> #### The code should run fine in any new Google colab project as long as the right uploads are done.


> ## *Please ignore this if you are working with Google collab.*

> ## Before you run this code, some installations need to be done:
---
> ### `!pip install numpy` to install numpy

> ### `!pip install sklearn` to install sklearn

> ### `!pip install Keras` to install keras

> ### `!pip install tensorflow` to install tensorflow , But ofcourse for model training its recommended you use the GPU version so install 

> ### `!pip install tensorflow-gpu` 

> ### `!pip install h5py` to install h5py

> ### `!pip install scipy` to install scipy

> ### `!pip install skimage` to install skimage

> ### `!pip install ffmpeg` to install ffmpeg

# **STEP 1)** 
> ## So First lets create the `trainer.npy` with the help of the videos and the datasets. Please upload the Avenue training dataset and set the directory location in the code.

---
> ## In the likely secenario where you cannot find the Avenue Dataset. 
> ## [Please look for it in this link.](http://www.cse.cuhk.edu.hk/leojia/projects/detectabnormal/dataset.html)


> ## So I shall uploading the data and keeping it in a folder called 
> ## `training_videos` and shall be copying the directory path into the code.
---



In [0]:
'''
Hello. Word of advice. Please ensure you check the variable video_source_path refers to the folder with the dataset of training 
and also make sure you have uploaded the correct training videos and not the testing videos


'''


from keras.preprocessing.image import img_to_array,load_img
import numpy as np
import glob
import os 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]



video_source_path='/content/training_videos'
fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it. 


	img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)



#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," training videos")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'

for video in videos:
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		images=os.listdir(framepath)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)


imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('trainer.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))
print("Program ended. Please wait while trainer.npy is created. \nRefresh when needed")
print('Number of frames created :', int(len(imagestore)))

Found  16  training videos
Program ended. Please wait while trainer.npy is created. 
Refresh when needed
Number of frames created : 227


 >## So right now a new model trainer called the `trainer.npy` should have been created! 

>## Please confirm its existence before you jump into the next section. 

---

# **STEP 2)**   
> ## Now that `trainer.npy` is created , we can run the below code and train the model using it.

> ## The model so created will be called `AnomalyDetector.h5`


In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
import numpy as np 
import argparse
from keras.layers import Conv3D,ConvLSTM2D,Conv3DTranspose
from keras.models import Sequential

''' The following load_model function code has been taken from 
Abnormal Event Detection in Videos using Spatiotemporal Autoencoder
by Yong Shean Chong Yong Haur Tay
Lee Kong Chian Faculty of Engineering Science, Universiti Tunku Abdul Rahman, 43000 Kajang, Malaysia.
It's main purpose is to help us generate the anomaly detector model
'''

#load_model starts here :----------------------------------------------------
def load_model():
	"""
	Return the model used for abnormal event 
	detection in videos using spatiotemporal autoencoder

	"""
	model=Sequential()
	model.add(Conv3D(filters=128,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
	model.add(Conv3D(filters=64,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))



	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))

	
	model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))


	model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))




	model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
	model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

	model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

	return model

#load_model ends here :----------------------------------------------------



X_train=np.load('trainer.npy')
frames=X_train.shape[2]
#Need to make number of frames divisible by 10 to ease the load_model


frames=frames-frames%10

X_train=X_train[:,:,:frames]
X_train=X_train.reshape(-1,227,227,10)
X_train=np.expand_dims(X_train,axis=4)
Y_train=X_train.copy()


epochs=200
batch_size=1



if __name__=="__main__":

	model=load_model()

	callback_save = ModelCheckpoint("AnomalyDetector.h5",
									monitor="mean_squared_error")

	callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

	print('Trainer has been loaded')
	model.fit(X_train,Y_train,
			  batch_size=batch_size,
			  epochs=epochs,
			  callbacks = [callback_save,callback_early_stopping]
			  )
print('Done\n Please wait while AnomalyDetector.h5 has been created \nRefresh when needed')





Trainer has been loaded
Epoch 1/200
22/22 [==============================] - 47s 2s/step - loss: 0.2402 - accuracy: 0.5296
Epoch 2/200


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


22/22 [==============================] - 47s 2s/step - loss: 0.2002 - accuracy: 0.5491
Epoch 3/200
22/22 [==============================] - 48s 2s/step - loss: 0.1813 - accuracy: 0.5735
Epoch 4/200
22/22 [==============================] - 48s 2s/step - loss: 0.1196 - accuracy: 0.6793
Epoch 5/200
22/22 [==============================] - 48s 2s/step - loss: 0.0880 - accuracy: 0.7185
Epoch 6/200
22/22 [==============================] - 48s 2s/step - loss: 0.0802 - accuracy: 0.7257
Epoch 7/200
22/22 [==============================] - 47s 2s/step - loss: 0.0757 - accuracy: 0.7315
Epoch 8/200
22/22 [==============================] - 47s 2s/step - loss: 0.0733 - accuracy: 0.7344
Epoch 9/200
22/22 [==============================] - 47s 2s/step - loss: 0.0716 - accuracy: 0.7353
Epoch 10/200
22/22 [==============================] - 48s 2s/step - loss: 0.0695 - accuracy: 0.7364
Epoch 11/200
22/22 [==============================] - 48s 2s/step - loss: 0.0685 - accuracy: 0.7372
Epoch 12/200
22/22 [

> ### **If this is taking too long , please feel free to take this already ready trained model**
>  ### It has been with epoches as 200. Its final accuracy stands at 0.7880
> # [Download it from here](https://drive.google.com/drive/folders/1AbF68y7ofutgZObca8D4K6Z33s66GDCP?usp=sharing)

> ## Please do not run the next code untill you can confirm that `AnomalyDetector.h5` has been successfully created and its most accurate version has been updated. It takes a while. You can come back a century later.


---
**Please note that for demonstration of training the model please reduce the epoches to lower values. Running high value of epoches on CPU is not recommended.**



---

> ## Right now there is no longer a compulsion to continue the execution in Google colab. The files ```trainer.npy``` and ```AnomalyDetector.h5``` are enough to enable testing the videos in CPU.


> ## However for the completion of this document as a full fledged project the code below can be used to run on testing videos 


---



#**STEP 3)** 
> ## Upload the "Avenue Dataset" testing Videos in the folder called the `testing_videos`

---
> ## For testing the videos we need to create a `tester.npy` and run the trained model on it


> ## Make sure you have the `AnomalyDetector.h5` in the main folder otherwise you shall ofcourse get some errors. 

In [0]:
'''
Hello. Word of advice. Please ensure you check the variable video_source_path refers to the folder with the dataset of testing videos
and also make sure you have uploaded the correct testing videos and not the training videos


'''


from keras.preprocessing.image import img_to_array,load_img
import numpy as np
import glob
import os 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]



video_source_path='/content/testing_videos'
fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it. 


	img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)
#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," testing videos")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'
total=0
video_count=0

for video in videos:
		video_count+=1
		print("Video number: ",video_count)
		print("Video:",str(video))
		image_count=0
		os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
		images=os.listdir(framepath)
		image_count=len(images)
		for image in images:
			image_path=framepath+ '/'+ image
			store(image_path)
		total=len(images)+total
		print("Number of images:",image_count,"\n----------\n")


imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('tester.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))

print("Program ended. All testing videos shall be stored in tester.npy \n Please wait while tester.npy is created. \nRefresh when needed")
print('Number of frames created :', int(total))
print ('Number of bunches=',int(total),"/10 = ",int(total/10))
print("\nCorrupted and unreadable bunches were ignored")

Found  22  testing videos
Video number:  1
Video: 03.avi
Number of images: 9 
----------

Video number:  2
Video: 01.avi
Number of images: 13 
----------

Video number:  3
Video: 05.avi
Number of images: 13 
----------

Video number:  4
Video: 15.avi
Number of images: 13 
----------

Video number:  5
Video: 02.avi
Number of images: 13 
----------

Video number:  6
Video: 13.avi
Number of images: 13 
----------

Video number:  7
Video: 16.avi
Number of images: 13 
----------

Video number:  8
Video: 09.avi
Number of images: 13 
----------

Video number:  9
Video: 10.avi
Number of images: 13 
----------

Video number:  10
Video: 17.avi
Number of images: 13 
----------

Video number:  11
Video: 04.avi
Number of images: 13 
----------

Video number:  12
Video: 06.avi
Number of images: 13 
----------

Video number:  13
Video: .ipynb_checkpoints
Number of images: 13 
----------

Video number:  14
Video: 07.avi
Number of images: 13 
----------

Video number:  15
Video: 20.avi
Number of images

#**STEP 4)** 
> ## Right now wait for the `tester.npy` to generate and then run the below code


> ## Some errors may creep in the dataset, but they will be removed in the program because they will be corrupted and unreadable

In [0]:

from keras.models import load_model
import numpy as np 




def mean_squared_loss(x1,x2):


	''' Compute Euclidean Distance Loss  between 
	input frame and the reconstructed frame'''


	diff=x1-x2
	a,b,c,d,e=diff.shape
	n_samples=a*b*c*d*e
	sq_diff=diff**2
	Sum=sq_diff.sum()
	dist=np.sqrt(Sum)
	mean_dist=dist/n_samples

	return mean_dist






'''Define threshold for Sensitivity
Lower the Threshhold,higher the chances that a bunch of frames will be flagged as Anomalous.

'''

#threshold=0.0004 #(Accuracy level 1)
#threshold=0.00042 #(Accuracy level 2)
threshold=0.0008#(Accuracy level Vishakha)

model=load_model('AnomalyDetector.h5')

X_test=np.load('tester.npy')
frames=X_test.shape[2]
#Need to make number of frames divisible by 10


flag=0 #Overall video flagq

frames=frames-frames%10

X_test=X_test[:,:,:frames]
X_test=X_test.reshape(-1,227,227,10)
X_test=np.expand_dims(X_test,axis=4)
counter =0
for number,bunch in enumerate(X_test):
	n_bunch=np.expand_dims(bunch,axis=0)
	reconstructed_bunch=model.predict(n_bunch)


	loss=mean_squared_loss(n_bunch,reconstructed_bunch)
	
	if loss>threshold:
		print("Anomalous bunch of frames at bunch number {}".format(number))
		counter=counter+1
		print("bunch number: ",counter)
		flag=1


	else:
		print('No anomaly')
		counter=counter+1
		print("bunch number: ",counter)



if flag==1:
	print("Anomalous Events detected")
else:
	print("No anomaly detected")
	
print("\nCorrupted and unreadable bunches were ignored")

No anomaly
bunch number:  1
No anomaly
bunch number:  2
No anomaly
bunch number:  3
Anomalous bunch of frames at bunch number 3
bunch number:  4
Anomalous bunch of frames at bunch number 4
bunch number:  5
No anomaly
bunch number:  6
No anomaly
bunch number:  7
No anomaly
bunch number:  8
No anomaly
bunch number:  9
No anomaly
bunch number:  10
No anomaly
bunch number:  11
No anomaly
bunch number:  12
No anomaly
bunch number:  13
No anomaly
bunch number:  14
No anomaly
bunch number:  15
No anomaly
bunch number:  16
No anomaly
bunch number:  17
Anomalous bunch of frames at bunch number 17
bunch number:  18
Anomalous bunch of frames at bunch number 18
bunch number:  19
Anomalous bunch of frames at bunch number 19
bunch number:  20
No anomaly
bunch number:  21
No anomaly
bunch number:  22
No anomaly
bunch number:  23
No anomaly
bunch number:  24
No anomaly
bunch number:  25
No anomaly
bunch number:  26
No anomaly
bunch number:  27
No anomaly
bunch number:  28
Anomalous Events detected


> ## Now to run the code on chosen files, we have to run the following code . Please set the video file location in the code

> ## Please upload the `test.mp4` or `test.avi` as a testing video. Please ensure that the video isnt doctored and edited. It should be continous stream of frames


---



In [0]:
'''
Hello. Word of advice. Please ensure you check the variable video_source_path refers to the folder with the dataset of training 
and also make sure you have uploaded the correct training videos and not the testing videos


'''
from keras.models import load_model
import numpy as np 

from keras.preprocessing.image import img_to_array,load_img
import numpy as np
import glob
import os 
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
import argparse
from PIL import Image
imagestore=[]



video_source_path='/content/'
fps=5
#fps refers to the number of seconds after which one frame will be taken . fps=5 means 1 frame after every 5 seconds. More like seconds per frame.

def create_dir(path):
	if not os.path.exists(path):
		os.makedirs(path)

def remove_old_images(path):
	filelist = glob.glob(os.path.join(path, "*.png"))
	for f in filelist:
		os.remove(f)

def store(image_path):
	img=load_img(image_path)
	img=img_to_array(img)


	#Resize the Image to (227,227,3) for the network to be able to process it. 


	img=resize(img,(227,227,3))

	#Convert the Image to Grayscale


	gray=0.2989*img[:,:,0]+0.5870*img[:,:,1]+0.1140*img[:,:,2]

	imagestore.append(gray)



#List of all Videos in the Source Directory.
videos=os.listdir(video_source_path)
print("Found ",len(videos)," files")


#Make a temp dir to store all the frames
create_dir(video_source_path+'/frames')

#Remove old images
remove_old_images(video_source_path+'/frames')

framepath=video_source_path+'/frames'
flag=0
for video in videos:
		if (video=="test.avi" or video=="test.mp4"):
			print("Test video found")
			flag=1
			os.system( 'ffmpeg -i {}/{} -r 1/{}  {}/frames/%03d.jpg'.format(video_source_path,video,fps,video_source_path))
			images=os.listdir(framepath)
			for image in images:
				image_path=framepath+ '/'+ image
				store(image_path)

if flag==0:
	print("Couldn't find test.mp4 or test.avi. Make sure you reupload and try this")
	exit()
imagestore=np.array(imagestore)
a,b,c=imagestore.shape
#Reshape to (227,227,batch_size)
imagestore.resize(b,c,a)
#Normalize
imagestore=(imagestore-imagestore.mean())/(imagestore.std())
#Clip negative Values
imagestore=np.clip(imagestore,0,1)
np.save('sample.npy',imagestore)
#Remove Buffer Directory
os.system('rm -r {}'.format(framepath))
print("Please wait while video is processed. \nRefresh when needed")


def mean_squared_loss(x1,x2):


	''' Compute Euclidean Distance Loss  between 
	input frame and the reconstructed frame'''


	diff=x1-x2
	a,b,c,d,e=diff.shape
	n_samples=a*b*c*d*e
	sq_diff=diff**2
	Sum=sq_diff.sum()
	dist=np.sqrt(Sum)
	mean_dist=dist/n_samples

	return mean_dist


'''Define threshold for Sensitivity
Lower the Threshhold,higher the chances that a bunch of frames will be flagged as Anomalous.

'''

#threshold=0.0004 #(Accuracy level 1)
#threshold=0.00042 #(Accuracy level 2)
threshold=0.0008#(Accuracy level 3)

model=load_model('AnomalyDetector.h5')

X_test=np.load('sample.npy')
frames=X_test.shape[2]
#Need to make number of frames divisible by 10


flag=0 #Overall video flagq

frames=frames-frames%10

X_test=X_test[:,:,:frames]
X_test=X_test.reshape(-1,227,227,10)
X_test=np.expand_dims(X_test,axis=4)
counter =0
for number,bunch in enumerate(X_test):
	n_bunch=np.expand_dims(bunch,axis=0)
	reconstructed_bunch=model.predict(n_bunch)


	loss=mean_squared_loss(n_bunch,reconstructed_bunch)
	
	if loss>threshold:
		print("Anomalous bunch of frames at bunch number {}".format(number))
		counter=counter+1
		print("bunch number: ",counter)
		flag=1


	else:
		print('No anomaly')
		counter=counter+1
		print("bunch number: ",counter)


print("----------------------------------------------------\nOUTPUT\n----------------------------------------------------\n")
if flag==1:
	print("Anomalous Events detected")
else:
	print("No anomaly detected")
	
print("\n----------------------------------------------------\nCorrupted and unreadable bunches were ignored")



Found  6  files
Test video found
Please wait while video is processed. 
Refresh when needed
Anomalous bunch of frames at bunch number 0
bunch number:  1
----------------------------------------------------
OUTPUT
----------------------------------------------------

Anomalous Events detected

----------------------------------------------------
Corrupted and unreadable bunches were ignored


# **WE ARE DONE**
> ## Yipee. That ends our project.
> ## Queries and comments shall be addressed later I guess.